In [1]:
import pulp as plp
from ALB_instance_tools import *
import numpy as np
import pandas as pd
import os

In [2]:
! pwd


/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW


In [3]:
'20_249.alb'.split(".")[0].split("_")[1]

'249'

In [4]:

#Gets list of all instance (.alb) files in the SALBP_benchmark/small\ data\ set_n\=20 folder
instance_list = get_instance_list('SALBP_benchmark/small data set_n=20')

#sorts instance list by instance number
instance_list = sorted(instance_list, key=lambda k: int(k['name'].split("_")[1]))


In [5]:
parse_alb(instance_list[517]['location'])

{'num_tasks': 20,
 'cycle_time': 1000,
 'order_strength': 0.289,
 'task_times': {'1': 187,
  '2': 30,
  '3': 88,
  '4': 202,
  '5': 107,
  '6': 240,
  '7': 96,
  '8': 251,
  '9': 173,
  '10': 238,
  '11': 156,
  '12': 86,
  '13': 80,
  '14': 93,
  '15': 218,
  '16': 221,
  '17': 70,
  '18': 46,
  '19': 67,
  '20': 166},
 'precedence_relations': [['1', '7'],
  ['2', '7'],
  ['3', '7'],
  ['4', '7'],
  ['6', '8'],
  ['6', '9'],
  ['7', '10'],
  ['7', '11'],
  ['7', '12'],
  ['7', '13'],
  ['8', '14'],
  ['8', '15'],
  ['12', '19'],
  ['13', '16'],
  ['14', '17'],
  ['15', '18'],
  ['16', '20']]}

In [6]:

def define_ALBP_1_problem(instance, max_stations = 20):
    prob = plp.LpProblem("ALPB_1", plp.LpMinimize)
    #creating decision variables
    tasks = plp.LpVariable.dicts("task_o_s", (instance['task_times'].keys(), range(1,max_stations + 1)), cat='Binary')
    #objective function
    prob += plp.lpSum([ station * tasks[task][station] for station in range(1,max_stations + 1) for task in instance['task_times'].keys()])
    #definining constraints
    #constraint 1 only choose 1 station for each task
    for task in instance['task_times'].keys():
        prob += plp.lpSum([tasks[task][station] for station in range(1,max_stations + 1)]) == 1
    #constraint 2 task and station assignment must respect takt time
    for station in range(1,max_stations + 1):
        prob += plp.lpSum([instance['task_times'][task] * tasks[task][station] for task in instance['task_times'].keys()]) <= instance['cycle_time']
    #constraint 3 tasks must respect precedence constraints
    for precedence in instance['precedence_relations']:
        prob += plp.lpSum([station * tasks[precedence[0]][station] for station in range(1,max_stations + 1)]) <= plp.lpSum([station * tasks[precedence[1]][station] for station in range(1,max_stations + 1)])
    return prob

def solve_ALBP_1(instance):
    prob = define_ALBP_1_problem(instance)
    prob.solve(solver=plp.XPRESS_PY( msg=False))
 
    return prob




In [9]:
def get_ALBP_solutions(problems_list):
    solutions = []
    for problem in problems_list:
        instance = parse_alb(problem['location'])
        prob = solve_ALBP_1(instance)
        #creates a new dictionary entry that contains the data on the instances
        print('solving problem', problem['name'])
        entry = {'name':problem['name']}
        entry['no_tasks'] = len(instance['task_times'].keys())
        entry['order_strength'] = instance['order_strength']
        entry['cycle_time'] = instance['cycle_time']
        max_station = -10
        for variable in prob.variables():
            
            if variable.varValue > 0:
                station = int(variable.name.split("_")[4])

                if station > max_station:
                    max_station = station
        entry['no_stations'] = max_station
        solutions.append(entry)
    return solutions
solution_outputs = get_ALBP_solutions(instance_list)

solving problem 20_1
solving problem 20_2
solving problem 20_3
solving problem 20_4
solving problem 20_5
solving problem 20_6
solving problem 20_7
solving problem 20_8
solving problem 20_9
solving problem 20_10
solving problem 20_11
solving problem 20_12
solving problem 20_13
solving problem 20_14
solving problem 20_15
solving problem 20_16
solving problem 20_17
solving problem 20_18
solving problem 20_19
solving problem 20_20
solving problem 20_21
solving problem 20_22
solving problem 20_23
solving problem 20_24
solving problem 20_25
solving problem 20_26
solving problem 20_27
solving problem 20_28
solving problem 20_29
solving problem 20_30
solving problem 20_31
solving problem 20_32
solving problem 20_33
solving problem 20_34
solving problem 20_35
solving problem 20_36
solving problem 20_37
solving problem 20_38
solving problem 20_39
solving problem 20_40
solving problem 20_41
solving problem 20_42
solving problem 20_43
solving problem 20_44
solving problem 20_45
solving problem 20_

In [12]:
solution_outputs = pd.DataFrame(solution_outputs)
solution_outputs.to_csv('ALBP_1_solutions.csv')

In [ ]:
instance['task_times'].keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'])

In [30]:
#Scoring functions
def task_time_weight(task, instance):
    return instance['task_times'][task]


     


#fills up each station with available tasks in order of score
def insert_task(tasks_dict, station_capacities, assignment_dict):
    for index, station in enumerate(station_capacities):
            for task in tasks_dict.keys():
                if instance['task_times'][task] <= station and all(predecessor not in tasks_dict.keys() for predecessor in tasks_dict[task]['predecessors']):
                    station_capacities[index] -= instance['task_times'][task]
                    assignment_dict[task] = index + 1
                    tasks_dict.pop(task)
                    return

# RA heuristic as described in "A comparative Evaluation of Heuristics for the Assembly Line Balancing Problem" by Ponnanbalam et. al              
def rank_and_assign(score_function, instance, max_stations = 20):
    station_capacities = [instance['cycle_time'] for i in range(0, max_stations)]
    tasks_dict = {}
    task_assignment = {}
    for task in instance['task_times'].keys():
        task_dict = {}
        task_dict['score'] = score_function(task, instance)
        task_dict['predecessors'] = [precedence[0] for precedence in instance['precedence_relations'] if precedence[1] == task]

        tasks_dict[task] = task_dict
    #sorts tasks_dict by score
    tasks_dict = {k: v for k, v in sorted(tasks_dict.items(), key=lambda item: item[1]['score'])}
    #Inserts tasks into stations until there are no more tasks
    while len(tasks_dict.keys()) > 0:
        insert_task(tasks_dict, station_capacities, task_assignment)
    return station_capacities, task_assignment

instance = parse_alb(instance_list[0]['location'])

station_capacities1, task_assignment1 = rank_and_assign(task_time_weight,instance)

In [38]:
#inserts tasks into first available station
def insert_task_iuff(tasks_dict, station_capacities, assignment_dict):
    for task in tasks_dict.keys():
        if tasks_dict[task]['available']:
            for index, station in enumerate(station_capacities):
                if instance['task_times'][task] <= station:
                    station_capacities[index] -= instance['task_times'][task]
                    assignment_dict[task] = index + 1
                    tasks_dict.pop(task)
            
            return

def update_tasks(tasks_dict, available_tasks, task_assignment):
    for task in tasks_dict.keys():
        if task not in available_tasks:
            if all(predecessor in task_assignment.keys() for predecessor in tasks_dict[task]['predecessors']):
                available_tasks.append(task)
 #TODO I need to make sure we are adding tasks in the right order, based on the score
    
     
# IUFF heuristic as described in "A comparative Evaluation of Heuristics for the Assembly Line Balancing Problem" by Ponnanbalam et. al              
def immediate_update_first_fit(score_function, instance, max_stations = 20):
    station_capacities = [instance['cycle_time'] for i in range(0, max_stations)]
    tasks_dict = {}
    task_assignment = {}
    available_tasks = []
    for task in instance['task_times'].keys():
        task_dict = {}
        task_dict['score'] = score_function(task, instance)
        task_dict['predecessors'] = [precedence[0] for precedence in instance['precedence_relations'] if precedence[1] == task]
        #adds tasks with no predecessors to available tasks
        if not task_dict['predecessors']:
            available_tasks.append(task)
        tasks_dict[task] = task_dict
    #sorts tasks_dict by score
    tasks_dict = {k: v for k, v in sorted(tasks_dict.items(), key=lambda item: item[1]['score'])}
    #Inserts first available tasks into a station

    while len(task_assignment) < instance['num_tasks']:
        insert_task_iuff(tasks_dict, station_capacities, task_assignment)
        update_tasks(tasks_dict, task_assignment)
    return station_capacities, task_assignment
instance = parse_alb(instance_list[0]['location'])
station_capacities, task_assignment = immediate_update_first_fit(task_time_weight,instance)

In [39]:
station_capacities

[20,
 9,
 89,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000]

In [36]:
task_assignment1

{'2': 1,
 '7': 1,
 '11': 1,
 '5': 1,
 '9': 1,
 '3': 1,
 '1': 1,
 '4': 1,
 '6': 2,
 '10': 2,
 '13': 2,
 '17': 1,
 '16': 2,
 '18': 2,
 '8': 3,
 '12': 3,
 '15': 3,
 '19': 3,
 '14': 3,
 '20': 3}

In [40]:
instance

{'num_tasks': 20,
 'cycle_time': 1000,
 'order_strength': 0.268,
 'task_times': {'1': 142,
  '2': 34,
  '3': 140,
  '4': 214,
  '5': 121,
  '6': 279,
  '7': 50,
  '8': 282,
  '9': 129,
  '10': 175,
  '11': 97,
  '12': 132,
  '13': 107,
  '14': 132,
  '15': 69,
  '16': 169,
  '17': 73,
  '18': 231,
  '19': 120,
  '20': 186},
 'precedence_relations': [['1', '6'],
  ['2', '7'],
  ['4', '8'],
  ['5', '9'],
  ['6', '10'],
  ['7', '11'],
  ['8', '12'],
  ['10', '13'],
  ['11', '13'],
  ['12', '14'],
  ['12', '15'],
  ['13', '16'],
  ['13', '17'],
  ['13', '18'],
  ['14', '20'],
  ['15', '19']]}

In [29]:
task_assignment

{'2': 1,
 '7': 1,
 '11': 1,
 '5': 1,
 '9': 1,
 '3': 1,
 '1': 1,
 '4': 1,
 '6': 2,
 '10': 2,
 '13': 2,
 '17': 1,
 '16': 2,
 '18': 2,
 '8': 3,
 '12': 3,
 '15': 3,
 '19': 3,
 '14': 3,
 '20': 3}

[['1', '7'],
 ['2', '7'],
 ['3', '7'],
 ['4', '7'],
 ['6', '8'],
 ['6', '9'],
 ['7', '10'],
 ['7', '11'],
 ['7', '12'],
 ['7', '13'],
 ['8', '14'],
 ['8', '15'],
 ['12', '19'],
 ['13', '16'],
 ['14', '17'],
 ['15', '18'],
 ['16', '20']]